In [7]:
import json
import numpy as np
from pathlib import Path
from scipy.io import loadmat
from opi.core import Calculator
from opi.input.structures.structure import Structure
from opi.input.simple_keywords import BasisSet, Method, Scf
from opi.input.blocks.block_loc import  BlockLoc

float_formatter = "{:.5f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})
config_dict = {
	"MOCoefficients": True,
	"1elPropertyIntegrals": ["dipole"],
	"1elIntegrals": ["H", "S"],
        "FockMatrix": ["F", "J", "K"],
	"Densities": ["scfp"],
	"JSONFormats": ["json"]
}

In [8]:
def utri2mat(utri: np.ndarray) -> np.ndarray:
    """Convert rolled-out upper triangle to full matrix"""
    n = int(-1 + np.sqrt(1 + 8*len(utri))) // 2
    iu1 = np.triu_indices(n)
    ret = np.empty((n, n))
    ret[iu1] = utri
    ret.T[iu1] = utri
    return ret

def trim_array(arr: np.ndarray) -> np.ndarray:
    """Trim a 1-D array to the last non-zero element"""
    if arr.ndim != 1:
        raise ValueError("Array must be 1-dimensional")
    return arr[:np.nonzero(arr)[0][-1]+1]

In [9]:
def read_orbnet(file: str | Path) -> tuple[int, np.ndarray]:
    """Extract some features from an OrbNet data file

    Returns
    -------
    nocc
        Number of valence occupied MOs
    f_diag
        Diagonal Fock matrix elements, occ + virt, each separately sorted in ascending order
    """
    diag = loadmat(str(file))['features_diag_ccpVTZ']
    nocc = diag.shape[0]
    f_occ_diag = diag[:,0]
    f_virt = utri2mat(diag[0,1:211])
    f_virt_diag = trim_array(f_virt.diagonal())
    f_diag = np.concatenate((np.sort(f_occ_diag), np.sort(f_virt_diag)))
    return nocc, f_diag

In [10]:
# read the orbnet data
orbnet_files = sorted(Path('orbnet_data').glob('*.mat'))
molnames = [p.stem.rsplit('_', 1)[0] for p in orbnet_files]
orbnet_data = {mol: read_orbnet(file) for mol, file in zip(molnames, orbnet_files)}

In [11]:
# ORCA parsing/processing functions
def orca_mos(orca_json: dict) -> np.ndarray:
    """Helper function to get MO coefficients from the ORCA JSON"""
    return np.column_stack([mo['MOCoefficients'] for mo in orca_json['Molecule']['MolecularOrbitals']['MOs']])

def orca_loc_orbwins(orca_output: Path) -> tuple[tuple[int, int] | None, tuple[int, int] | None]:
    """Read the valence occupied and valence virtual localized orbital windows from the ORCA output file"""
    occ = None
    virt = None
    with open(orca_output) as f:
        for line in f:
            if not occ and 'Orbital range for localization' in line:
                fields = line.split()
                occ = int(fields[-3]), int(fields[-1])
                continue
            if not virt and 'Orbital range for VVO localization' in line:
                fields = line.split()
                virt = int(fields[-3]), int(fields[-1])
                break
    return occ, virt

def orca_process(outfile: Path, jsonfile: Path) -> tuple[int, np.ndarray]:
    """Extract some features from ORCA output and JSON files

    Returns
    -------
    nocc
        Number of valence occupied MOs
    f_diag
        Diagonal Fock matrix elements, occ + virt, each separately sorted in ascending order
    """
    # read orbital windows from the output
    orbwin_occ, orbwin_vvo = orca_loc_orbwins(outfile)
    nocc = orbwin_occ[1] - orbwin_occ[0] + 1
    nvvo = orbwin_vvo[1] - orbwin_vvo[0] + 1
    # read ORCA JSON file
    with open(jsonfile) as f:
        json_data = json.load(f)
    # localized MO coefficients
    c = orca_mos(json_data)
    # trim to valence + VVO
    c_val = c[:,orbwin_occ[0]:orbwin_vvo[1]+1]
    # one-electron Hamiltonian
    h = np.array(json_data['Molecule']['H-Matrix'])
    # two-electron Fock matrix (alpha)
    g = np.array(json_data['Molecule']['F-Matrix'][0])
    # total Fock matrix
    fao = h + g
    # transform to localized MOs
    fmo = c_val.T @ fao @ c_val
    f_diag = fmo.diagonal()
    # sort occ and virt elements
    f_diag = np.concatenate((np.sort(f_diag[:nocc]), np.sort(f_diag[nocc:])))
    return nocc, f_diag

def orca_run(xyz_file: Path) -> None:
    """Run an ORCA quantum chemistry calculation from a given XYZ file.

    Parameters
    ----------
    xyz_file : Path
        Path to the input .xyz file. A directory with the same stem name will be created
        in the same location to store ORCA outputs.
    """
    xyz_file = Path(xyz_file)
    mol_name = xyz_file.stem
    working_dir = Path("RUN")
    calc = Calculator(basename=mol_name, working_dir=working_dir)
    structure = Structure.from_xyz(xyz_file)
    calc.structure = structure
    calc.structure.charge = 0
    calc.structure.multiplicity = 1
    calc.input.add_simple_keywords(
        Scf.TIGHTSCF,
        Scf.NOSLOPPYSCFCHECK,
        Scf.PMODEL,
        BasisSet.CC_PVTZ,
        Method.HF,
    )
    calc.input.add_blocks(
        BlockLoc(locmet='newboys', locmetvirt='livvo', tol= 1e-8, occ=True, virt=True, iaobasis='minao_auto_pp')
    )
    runner = calc._create_runner()
    runner.create_gbw_json(mol_name,config=config_dict)
    calc.write_input()
    print("Running ORCA calculation ...", end="")
    calc.run()
    print("   Done")

In [12]:
orca_run(Path("./RUN/water.xyz"))
orca_run(Path("./RUN/butane.xyz"))
orca_run(Path("./RUN/ethane.xyz"))
orca_run(Path("./RUN/isobutane.xyz"))
orca_run(Path("./RUN/methane.xyz"))
orca_run(Path("./RUN/propane.xyz"))

Running ORCA calculation ...   Done
Running ORCA calculation ...   Done
Running ORCA calculation ...   Done
Running ORCA calculation ...   Done
Running ORCA calculation ...   Done
Running ORCA calculation ...   Done


In [13]:
# read the ORCA data
xyz_files = list(Path("RUN").glob("*.xyz"))
molnames = [f.stem for f in xyz_files]
base_dir = Path("RUN")
orca_outfiles = [base_dir / f"{mol}.out" for mol in molnames]
orca_jsonfiles = [base_dir / f"{mol}.json" for mol in molnames]
orca_data = {mol: orca_process(outfile, jsonfile) for mol, outfile, jsonfile in zip(molnames, orca_outfiles, orca_jsonfiles)}

In [14]:
# compare
for mol in molnames:
    nocc_orbnet, fdiag_orbnet = orbnet_data[mol]
    nocc_orca, fdiag_orca = orca_data[mol]
    if nocc_orca != nocc_orbnet:
        print(f'{mol}: {nocc_orca=} != {nocc_orbnet=}')
    elif not np.allclose(fdiag_orca, fdiag_orbnet):
        if not np.allclose(fdiag_orca[:nocc_orca], fdiag_orbnet[:nocc_orbnet]):
            maxdiff = np.max(np.abs(fdiag_orca[:nocc_orca] - fdiag_orbnet[:nocc_orbnet]))
            print(f'{mol}: Focc differs by {maxdiff}!')
        if not np.allclose(fdiag_orca[nocc_orca:], fdiag_orbnet[nocc_orbnet:]):
            maxdiff = np.max(np.abs(fdiag_orca[nocc_orca:] - fdiag_orbnet[nocc_orbnet:]))
            print(f'{mol}: Fvirt differs by {maxdiff}!')
    else:
        print(f'{mol}: OK!')

ethane: Focc differs by 0.3319560217963802!
ethane: Fvirt differs by 0.4311658656146292!
isobutane: Focc differs by 0.3958425907775406!
isobutane: Fvirt differs by 0.436545883685849!
propane: Focc differs by 0.368898752543224!
propane: Fvirt differs by 0.4326396049755893!
water: Focc differs by 0.4515125716353008!
water: Fvirt differs by 0.316564239132194!
butane: Focc differs by 0.38690507898367843!
butane: Fvirt differs by 0.4317966851761648!
methane: Focc differs by 0.29617198001750067!
methane: Fvirt differs by 0.41635311707811784!
